## Advanced Cleaning of Steam Games Data

In this notebook we try out two different recommenders on [Steam dataset on Kaggle](https://www.kaggle.com/tamber/steam-video-games). We will use:
- ```numpy, pandas, sklearn``` for data preprocessing

The dataset has no header but comes in 5 columns:
- User ID: integer
- Game title: string
- Activity: string (purchase/play)
- Status: float (1.0 if activity is purchase, total number of hours in game if activity is play)
- A column full of 0's, will discard

In [1]:
import pandas as pd
import numpy as np 
import matplotlib
# import matplotlib.pyplot as plt 
import seaborn as sns
import turicreate
import sklearn as sk
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

In [96]:
games = pd.read_csv('steam-200k.csv')
games = games.rename({'151603712':'userId', 'The Elder Scrolls V Skyrim': 'gameName', '1.0':'Actions'}, axis = 1)
games.drop(['0'],axis = 1, inplace = True)
games.head()

,userId,gameName,purchase,Actions
0,151603712,The Elder Scrolls V Skyrim,play,273.0
1,151603712,Fallout 4,purchase,1.0
2,151603712,Fallout 4,play,87.0
3,151603712,Spore,purchase,1.0
4,151603712,Spore,play,14.9


### get a sense of the data by displaying some basic properties

In [95]:
games.info()
print("number of distinct users = %d" %games['userId'].nunique()) 
print("number of distinct games = %d" %games['gameName'].nunique()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199999 entries, 0 to 199998
Data columns (total 4 columns):
userId      199999 non-null int64
gameName    199999 non-null object
purchase    199999 non-null object
Actions     199999 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.1+ MB
number of distinct users = 12393
number of distinct games = 5155


According to the description of the dataset, when the value under purchase column is equal to 'purchase', the Actions will always be 1.0. Therefore, those information is considered as redundant and could be nicely cleaned if we split the purchase column into two individual columns representing 'purchase' and 'play'

In [99]:
# split the purchase column to two dataframes and perform an outer join to group highly duplicated row
games_temp = games[games['purchase'] == 'play']
games_temp =games_temp.rename({'purchase':'play'}, axis = 1)
games = games[games.purchase =='purchase']
games.drop(columns = 'Actions', inplace = True)
result = pd.merge(games,games_temp, how='outer', on=['userId','gameName'])
# reindex to group the data associated with the same user together
reindex_result = result.sort_values(by = 'userId')
reindex_result.set_index(np.arange(len(reindex_result.index)))
reindex_result.purchase.replace(['purchase'], [1], inplace=True)
reindex_result.play.replace(['play'], [1], inplace=True)
# check whether the data has been fully merged 
check= reindex_result[reindex_result.purchase != 1]
print(check)

           userId                    gameName  purchase  play  Actions
129534  151603712  The Elder Scrolls V Skyrim       NaN   1.0    273.0


This is very likely an input mistake as it is the only occasion when a game is played but not yet purchased by the user.  we will manually change purchase value to 1 in this case

In [103]:
reindex_result.loc[129534,'purchase'] = 1
reindex_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129535 entries, 41879 to 83172
Data columns (total 5 columns):
userId      129535 non-null int64
gameName    129535 non-null object
purchase    129535 non-null float64
play        70785 non-null float64
Actions     70785 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 10.9+ MB


now we have finish the basic data cleaning.  We could take a deeper look into the data and start some advanced data training to prepare for our recommender system

In [107]:
data_copy = reindex_result

 Again, we can easily get basic information about the data in each column 

In [108]:
data_copy.describe()

,userId,purchase,play,Actions
count,1.295350e+05,129535.0,70785.0,70785.000000
mean,1.024410e+08,1.0,1.0,48.770761
std,7.236206e+07,0.0,0.0,228.927258
min,5.250000e+03,1.0,1.0,0.100000
25%,4.548346e+07,1.0,1.0,1.000000
50%,8.605570e+07,1.0,1.0,4.500000
75%,1.542307e+08,1.0,1.0,19.100000
max,3.099031e+08,1.0,1.0,11754.000000


### Some obervations

Above summary shows that ```purchase``` has only a single value 1.0. We will **not** include purchase status in building our recommender, for two reasons:
1. number of hours played is more interesting to look at - it is an implicit feedback of the user's preference for games he/she has purchased
2. if using [Jaccard similarity](https://apple.github.io/turicreate/docs/api/generated/turicreate.recommender.item_similarity_recommender.ItemSimilarityRecommender.html) when measuring the similarity between two sets of elements, the number of hours played are treated as binary purchase status (1/0)

For ```play``` column, the data is heavily skewed and has a large range. There are different ways to do normalization. Since we are going to use it to represent the user's ranking of preference within his / her own purchases, let's convert it to the percentage of hours each user spends on each game he / she owns. 

But before doing that, notice there are 128804 purchase records and only 70489 play records, meaning some users bought certain games but never played them. We need to find a score for those less-favoured games too. To differentiate them from the games that a user has never bought, we can impute the corresponding play fields with a small value less than the min 0.1, say 0.05. Then we can move on to calculate the percentage as planned.

In [109]:
data_copy.drop("purchase", axis = 1, inplace = True)
data_copy["Actions"].fillna(0.05, inplace=True)
data_copy["play"].fillna(0, inplace=True)


just in case there is duplicated row in the dataframe, we implement the following function to ensure that every row in our dataframe will be distinct

In [110]:
#remove duplicates 
data_copy = data_copy.drop_duplicates(subset=None,keep="first")

besides, we could also have duplicated rows with the same userId and gameName appearing together. This might be some mistakes that happen during the data collection stage so that the hours has not been fully merged and accumulated. we wanna sum up the number of hours for the corrections

In [111]:
dp = data_copy[data_copy.duplicated(subset=['userId','gameName'], keep=False)]
dp = dp.groupby(['userId','gameName'], as_index=False)['Actions'].sum()
print(dp)
dp = dp.groupby(['userId','gameName'], as_index=False)['Actions'].sum()
data_copy.update(dp)
data_copy = data_copy.drop_duplicates(subset=['userId','gameName'],keep="first")

       userId                                      gameName  Actions
0    28472068                          Grand Theft Auto III      0.5
1    28472068                  Grand Theft Auto San Andreas      0.9
2    28472068                    Grand Theft Auto Vice City      5.7
3    33865373                   Sid Meier's Civilization IV    137.0
4    50769696                  Grand Theft Auto San Andreas     14.0
5    59925638                       Tom Clancy's H.A.W.X. 2      7.4
6    71411882                          Grand Theft Auto III      1.3
7    71510748                  Grand Theft Auto San Andreas      0.8
8   118664413                  Grand Theft Auto San Andreas      2.1
9   148362155                  Grand Theft Auto San Andreas     26.3
10  176261926                   Sid Meier's Civilization IV     14.8
11  176261926  Sid Meier's Civilization IV Beyond the Sword    564.4


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[col] = expressions.where(mask, this, that)


Replace actual hours played with percentage of hours on a game for each user

In [112]:
d = data_copy.groupby('userId')['Actions'].apply(lambda x: x/ x.sum())
data_copy.update(d)
# convert userId to int
data_copy['userId'] = data_copy['userId'].astype(int)

### Output and save the dataframe

In [113]:
data_copy.to_pickle('clean_steam.pkl')

### Before building the recommender, here is an optional step - indexing the game names with integers
Based on doc of turicreate.recommender.create, the user ID and item ID columns can be either int or str, meaning that keeping the game title as is should be fine. Here let's do an extra step that indexes those strings with integers. (Maybe when we feel less lazy we can try spark.ml's recommender on it too.)

However indexing is essential when you build models with Spark's machine learning libraries.

In [114]:
data_copy['gameId']= data_copy.gameName.astype('category').cat.codes

### we want to create a look up table for the gameName and gameId columns

In [115]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 128792 entries, 41879 to 83172
Data columns (total 5 columns):
userId      128792 non-null int64
gameName    128792 non-null object
play        128792 non-null float64
Actions     128792 non-null float64
gameId      128792 non-null int16
dtypes: float64(2), int16(1), int64(1), object(1)
memory usage: 5.2+ MB


In [129]:
game_map = data_copy[['gameName','gameId']].copy().drop_duplicates().sort_values(by=['gameId'])
game_map.head()
data_copy.to_pickle('gameMap.pkl')
export_csv = game_map.to_csv ('gameList.csv', index = None, header=True)

### Item - Item similarity model in Turicreate

According to the documentation [here](https://apple.github.io/turicreate/docs/userguide/recommender/using-trained-models.html), turicreate excludes the items that are observed for each user. In order to test the accuracy of the model, it is important to split a proportion of user's data into train and test for validation. 
but for now, let us just feed all the data in to our model.

There are 3 similarity measurements for similarity type - jaccard, cosine and pearson. In our case, We choose Pearson to measure the loss.

In [130]:
# create SFrame from DataFrame
from turicreate import SFrame

game_sf = SFrame(data=data_copy)

item_item_reco = turicreate.recommender.item_similarity_recommender.create( \
    game_sf, user_id='userId', item_id='gameId', \
    target="Actions", similarity_type='pearson')

Warning: Ignoring columns gameName, play;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 128792 observations with 12393 users and 5155 items.

Data prepared in: 0.132695s

...

| 465.392ms                           | 100              | 5155            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.520646s

In [131]:
# Get the k most similar items for each item in items. Default number is 10.
similar_games = item_item_reco.get_similar_items()

now We have created the model. Turicreate provides a nice functionality for users to check the k most similar items of an input. We would like to expriment our model with a few sample inputs.

In [138]:
# for example to check some results
game_map_sframe = turicreate.SFrame(game_map)

similar_games_named = similar_games.join(game_map_sframe, on="gameId", how="left") \
   .join(game_map_sframe, on={"similar":"gameId"}, how="left") \
  .rename({"gameName.1":"similar_game"}, True) \
   .select_columns(["gameId", "gameName", "similar", "similar_game", "score", "rank"])

# print the result of one game for testing
a =similar_games_named[similar_games_named['gameName']== "Half-Life 2 Episode One"]
a.print_rows(num_rows=10, num_columns=6) 


+--------+-------------------------+---------+-------------------------------+
| gameId |         gameName        | similar |          similar_game         |
+--------+-------------------------+---------+-------------------------------+
|  2072  | Half-Life 2 Episode One |   2070  |          Half-Life 2          |
|  2072  | Half-Life 2 Episode One |   2077  |  Half-Life Deathmatch Source  |
|  2072  | Half-Life 2 Episode One |   2073  |    Half-Life 2 Episode Two    |
|  2072  | Half-Life 2 Episode One |   2074  |     Half-Life 2 Lost Coast    |
|  2072  | Half-Life 2 Episode One |   3627  |       SEGA Bass Fishing       |
|  2072  | Half-Life 2 Episode One |   1003  |           Crazy Taxi          |
|  2072  | Half-Life 2 Episode One |   480   | Battlefield Bad Company 2 ... |
|  2072  | Half-Life 2 Episode One |   3222  |             Portal            |
|  2072  | Half-Life 2 Episode One |   2076  |      Half-Life Blue Shift     |
|  2072  | Half-Life 2 Episode One |   1073  |      

In [149]:
# print the result of one game for testing
b =similar_games_named[similar_games_named['gameName']==  'NBA 2K12']
b.print_rows(num_rows=10, num_columns=6) 

+--------+----------+---------+-----------------------------+
| gameId | gameName | similar |         similar_game        |
+--------+----------+---------+-----------------------------+
|  2848  | NBA 2K12 |   3043  | Out of the Park Baseball 14 |
|  2848  | NBA 2K12 |   2581  |           MLB 2K10          |
|  2848  | NBA 2K12 |   2582  |           MLB 2K11          |
|  2848  | NBA 2K12 |   5080  |             Ys I            |
|  2848  | NBA 2K12 |   4386  |        The Golf Club        |
|  2848  | NBA 2K12 |   2853  |           NBA 2K9           |
|  2848  | NBA 2K12 |   2584  |   MLB Front Office Manager  |
|  2848  | NBA 2K12 |   3789  |       Shattered Union       |
|  2848  | NBA 2K12 |   866   |         CivCity Rome        |
|  2848  | NBA 2K12 |   2311  | Jade Empire Special Edition |
+--------+----------+---------+-----------------------------+
+---------------------+------+
|        score        | rank |
+---------------------+------+
|  0.5828672647476196 |  1   |
|  0.511

It seems the result we get is actually reasonable.  The model is able to find games in similar genres, and sometimes even the same game of other versions. This proves that our model we build is able to capture the relationships embedded behind the mappings of items and users.

### Let's pick a user and see what recommendations he / she gets:

In [43]:
rec_result = item_item_reco.recommend(diversity=1,random_seed=0).join(game_map_sframe, on="gameId", how="left")
print (rec_result)

recommendations finished on 1000/12393 queries. users per second: 17640.7

recommendations finished on 2000/12393 queries. users per second: 19962.7

recommendations finished on 3000/12393 queries. users per second: 21226.9

recommendations finished on 4000/12393 queries. users per second: 21591.2

recommendations finished on 5000/12393 queries. users per second: 21770.6

...

recommendations finished on 9000/12393 queries. users per second: 21953.2

recommendations finished on 10000/12393 queries. users per second: 21913.9

recommendations finished on 11000/12393 queries. users per second: 21565.8

recommendations finished on 12000/12393 queries. users per second: 21858.8

+--------+--------+-------+------+--------------------------------+
| userId | gameId | score | rank |            gameName            |
+--------+--------+-------+------+--------------------------------+
|  5250  |  5061  |  1.0  |  1   |          Xpand Rally           |
|  5250  |  3364  |  1.0  |  2   |            RECYCLE             |
|  5250  |  1324  |  1.0  |  3   |     Don Bradman Cricket 14     |
|  5250  |  838   |  1.0  |  4   |         Choplifter HD          |
|  5250  |  2955  |  1.0  |  5   | Nobunaga's Ambition Souzou...  |
|  5250  |  1271  |  1.0  |  6   |          Diaper Dash           |
|  5250  |  4934  |  1.0  |  7   |     Warrior Kings Battles      |
|  5250  |  2265  |  1.0  |  8   |            Insane 2            |
|  5250  |  3600  |  1.0  |  9   | Rugby League Team Manager 2015 |
|  5250  |  1705  |  1.0  |  10  |    Fast & Furious Showdown     |
+--------+--------+-------+------+--------------------------------+
[123930 rows x 5 columns]
Note: Only the head of

### In order to better estimate the accuracy of our model, we decide to split the training and testing set, and use "precision_recall" as our metric. 
check [here](https://apple.github.io/turicreate/docs/api/generated/turicreate.recommender.item_similarity_recommender.ItemSimilarityRecommender.evaluate.html#turicreate.recommender.item_similarity_recommender.ItemSimilarityRecommender.evaluate) for more info.

In [16]:
game_sf2 = SFrame(data=data_copy)

#The test dataset is generated by first choosing max_num_users out of the total number of users in dataset. Then, for each of the chosen test users, 
#a portion of the user’s items (determined by item_test_proportion) is randomly chosen to be included in the test set. 
#This split allows the training data to retain enough information about the users in the testset, so that adequate recommendations can be made. 
#The total number of users in the test set may be fewer than max_num_users if a user was chosen for the test set but none of their items are selected.

train_sframe, test_sframe = turicreate.recommender.util.random_split_by_user(game_sf2, user_id="userId", item_id="gameId", max_num_users=500,item_test_proportion=0.2,random_seed = 0)

In [53]:
item_item_reco = turicreate.recommender.item_similarity_recommender.create( \
    game_sf2, user_id='userId', item_id='gameId', \
    target="Actions", similarity_type='pearson')
rec = item_item_reco.recommend(k= 50, diversity=2,random_seed=1)
rec = rec.join(game_map_sframe, on="gameId", how="left") 
from turicreate.toolkits.recommender.util import precision_recall_by_user
result = precision_recall_by_user(test_sframe, rec)

Warning: Ignoring columns gameName, play;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 128792 observations with 12393 users and 5155 items.

Data prepared in: 0.137561s

...

recommendations finished on 8000/12393 queries. users per second: 6982.62

recommendations finished on 9000/12393 queries. users per second: 6969.45

recommendations finished on 10000/12393 queries. users per second: 6987.31

recommendations finished on 11000/12393 queries. users per second: 7083.76

recommendations finished on 12000/12393 queries. users per second: 7195.74

In [46]:
#look at the data of user 858433 in recommendation

# Filtering
filter_sf =rec[(rec['userId']==  8542204)] 

# Displaying
(filter_sf[['userId','gameId','gameName','score']]). print_rows(num_rows=50, num_columns=4)

+---------+--------+--------------------------------+---------------------+
|  userId | gameId |            gameName            |        score        |
+---------+--------+--------------------------------+---------------------+
| 8542204 |  1607  |        FIFA Manager 10         |         1.0         |
| 8542204 |  2306  |           Jack Keane           |         1.0         |
| 8542204 |  1705  |    Fast & Furious Showdown     |         1.0         |
| 8542204 |  3848  |         SimpleRockets          |         1.0         |
| 8542204 |  831   |       Chip's Challenge 2       |         1.0         |
| 8542204 |  3364  |            RECYCLE             |         1.0         |
| 8542204 |  2265  |            Insane 2            |         1.0         |
| 8542204 |  1324  |     Don Bradman Cricket 14     |         1.0         |
| 8542204 |  3600  | Rugby League Team Manager 2015 |         1.0         |
| 8542204 |  838   |         Choplifter HD          |         1.0         |
| 8542204 | 

In [47]:
# Filtering
test_sf = test_sframe[(test_sframe['userId']==   8542204 )] 
# Displaying
test_sf.materialize()
(test_sf[['userId', 'gameName', 'gameId','Actions']]). print_rows(num_rows=21, num_columns=4)

+---------+------------------------------+--------+-----------------------+
|  userId |           gameName           | gameId |        Actions        |
+---------+------------------------------+--------+-----------------------+
| 8542204 |          Metro 2033          |  2712  |  0.001405728342997716 |
| 8542204 | The Witcher Enhanced Edition |  4527  | 0.0001757160428747145 |
| 8542204 |      Grand Theft Auto V      |  1978  |   0.0460376032331752  |
| 8542204 |           The Crew           |  4343  |  0.008434370057986295 |
| 8542204 |    Call of Duty Black Ops    |  726   |  0.007380073800738009 |
| 8542204 |           Portal 2           |  3223  |  0.05622913371990863  |
| 8542204 |     ARK Survival Evolved     |   84   | 0.0059743454577402925 |
| 8542204 |        Torchlight II         |  4658  | 0.0035143208574942896 |
| 8542204 |       Sniper Elite V2        |  3898  | 0.0035143208574942896 |
+---------+------------------------------+--------+-----------------------+
[9 rows x 4 

In [48]:
# Filtering
train_sf = train_sframe[(train_sframe['userId']==   8542204 )] 
# Displaying
train_sf.materialize()
(train_sf[['userId', 'gameName', 'gameId','Actions']]). print_rows(num_rows=35, num_columns=4)

+---------+-------------------------------+--------+-----------------------+
|  userId |            gameName           | gameId |        Actions        |
+---------+-------------------------------+--------+-----------------------+
| 8542204 |             Dota 2            |  1336  |   0.4533473906167634  |
| 8542204 |           Dead Space          |  1152  | 0.0010542962572482868 |
| 8542204 |      Natural Selection 2      |  2901  |  0.001405728342997716 |
| 8542204 |   Call of Duty Black Ops II   |  730   | 0.0017571604287471448 |
| 8542204 |         Left 4 Dead 2         |  2475  | 0.0017571604287471448 |
| 8542204 | Microsoft Flight Simulator... |  2721  |  0.000702864171498858 |
| 8542204 | Call of Duty Advanced Warfare |  724   | 0.0001757160428747145 |
| 8542204 |             Arma 2            |  322   |  0.000702864171498858 |
| 8542204 |   Arma 2 Operation Arrowhead  |  326   |  0.000351432085749429 |
| 8542204 | The Witcher 2 Assassins of... |  4522  | 0.0021085925144965737 |

In [49]:
# Filtering
total_sf = game_sf2[(game_sf2['userId']==   8542204 )] 
# Displaying
total_sf.materialize()
(total_sf[['userId', 'gameName', 'gameId','Actions']]). print_rows(num_rows=44, num_columns=4)

+---------+-------------------------------+--------+-----------------------+
|  userId |            gameName           | gameId |        Actions        |
+---------+-------------------------------+--------+-----------------------+
| 8542204 |             Dota 2            |  1336  |   0.4533473906167634  |
| 8542204 |           Dead Space          |  1152  | 0.0010542962572482868 |
| 8542204 |           Metro 2033          |  2712  |  0.001405728342997716 |
| 8542204 |      Natural Selection 2      |  2901  |  0.001405728342997716 |
| 8542204 |   Call of Duty Black Ops II   |  730   | 0.0017571604287471448 |
| 8542204 |         Left 4 Dead 2         |  2475  | 0.0017571604287471448 |
| 8542204 | Microsoft Flight Simulator... |  2721  |  0.000702864171498858 |
| 8542204 | Call of Duty Advanced Warfare |  724   | 0.0001757160428747145 |
| 8542204 |             Arma 2            |  322   |  0.000702864171498858 |
| 8542204 |   Arma 2 Operation Arrowhead  |  326   |  0.000351432085749429 |

In [21]:
# obviously for those users the precision will be 0 because there is no data about those users' in testsframe

In [22]:
result = test_sframe.join(result, on="userId", how="left") \
   .select_columns(["userId","gameId", "gameName", "precision", "recall", "count"])

In [23]:
result.print_rows(num_rows=100, num_columns=6)

+----------+--------+-------------------------------+-----------+--------+-------+
|  userId  | gameId |            gameName           | precision | recall | count |
+----------+--------+-------------------------------+-----------+--------+-------+
| 1024319  |  2078  |    Half-Life Opposing Force   |    0.0    |  0.0   |   1   |
| 1364546  |  1179  |       Deathmatch Classic      |    0.0    |  0.0   |   1   |
| 4325465  |  1179  |       Deathmatch Classic      |    0.0    |  0.0   |   1   |
| 8542204  |  2712  |           Metro 2033          |    0.0    |  0.0   |   9   |
| 8542204  |  4527  |  The Witcher Enhanced Edition |    0.0    |  0.0   |   9   |
| 8542204  |  1978  |       Grand Theft Auto V      |    0.0    |  0.0   |   9   |
| 8542204  |  4343  |            The Crew           |    0.0    |  0.0   |   9   |
| 8542204  |  726   |     Call of Duty Black Ops    |    0.0    |  0.0   |   9   |
| 8542204  |  3223  |            Portal 2           |    0.0    |  0.0   |   9   |
| 85

In [24]:
# Filtering
nonzero_sframe = result[(result['recall']!=  0.0)] 
nonzero_sframe.materialize()
print(nonzero_sframe)

# this is weird. we need more investigation

+--------+--------+----------+-----------+--------+
| userId | gameId | gameName | precision | recall |
+--------+--------+----------+-----------+--------+
+--------+--------+----------+-----------+--------+
+-------+
| count |
+-------+
+-------+
[0 rows x 6 columns]



 ### let's try the item_similarity_recommender models which recommends the k highest scored items based on the interactions given in observed_items

In [85]:
item_item_reco = turicreate.recommender.item_similarity_recommender.create( \
    game_sf2, user_id='userId', item_id='gameId', \
    target="Actions", similarity_type='pearson')

rec2 = item_item_reco.recommend_from_interactions(observed_items = [2072], items =game_map_sframe['gameId'], k= 50, diversity=2,random_seed=1)
rec2 = rec2.join(game_map_sframe, on="gameId", how="left") 
rec2.sort('rank',ascending = True).print_rows(num_rows=50, num_columns=4) 


Warning: Ignoring columns gameName, play;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 128792 observations with 12393 users and 5155 items.

Data prepared in: 0.135404s

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.703756s

+--------+---------------------+------+--------------------------------+
| gameId |        score        | rank |            gameName            |
+--------+---------------------+------+--------------------------------+
|  4459  |         1.0         |  1   |       The Promised Land        |
|  3364  |         1.0         |  2   |            RECYCLE             |
|  1607  |         1.0         |  3   |        FIFA Manager 10         |
|  1271  |         1.0         |  4   |          Diaper Dash           |
|  4042  |         1.0         |  5   |      Stargate Resistance       |
|  2265  |         1.0         |  6   |            Insane 2            |
|  2955  |         1.0         |  7   | Nobunaga's Ambition Souzou...  |
|  813   |         1.0         |  8   |   Championship Manager 2010    |
|  2862  |  0.975877192982456  |  9   | NOBUNAGA'S AMBITION Kakush...  |
|  2763  |         0.75        |  10  |           Monochroma           |
|  2535  |  0.6965559095736611 |  11  | London 2012

In [75]:
print(train_sf)

+---------+-------------------------------+------+-----------------------+--------+
|  userId |            gameName           | play |        Actions        | gameId |
+---------+-------------------------------+------+-----------------------+--------+
| 8542204 |             Dota 2            | 1.0  |   0.4533473906167634  |  1336  |
| 8542204 |           Dead Space          | 1.0  | 0.0010542962572482868 |  1152  |
| 8542204 |      Natural Selection 2      | 1.0  |  0.001405728342997716 |  2901  |
| 8542204 |   Call of Duty Black Ops II   | 1.0  | 0.0017571604287471448 |  730   |
| 8542204 |         Left 4 Dead 2         | 1.0  | 0.0017571604287471448 |  2475  |
| 8542204 | Microsoft Flight Simulator... | 1.0  |  0.000702864171498858 |  2721  |
| 8542204 | Call of Duty Advanced Warfare | 0.0  | 0.0001757160428747145 |  724   |
| 8542204 |             Arma 2            | 1.0  |  0.000702864171498858 |  322   |
| 8542204 |   Arma 2 Operation Arrowhead  | 1.0  |  0.000351432085749429 |  